## Community Health Equity Lab: New York COVID19 Response Analysis

In [9]:
import pandas as pd
import numpy as np
import warnings
from plotnine import *
#import bamboolib as bam

from sklearn.model_selection import train_test_split # simple TT split cv

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

pd.options.display.max_columns = 300
pd.set_option('display.max_rows', 500)

## Data

In [10]:
#reading zip code data and joining borough name
ny = pd.read_csv('data/ny18_cleaned.csv')
ny.head()

,GEOID_ZIP,ALAND10,AWATER10,tot_pop,white,per_white,black,per_black,native,per_native,asian,per_asian,nativehawaiian,per_hawaiian,other,per_other,two_or_mor,per_two_or,male,female,under_5,per_under_5,5_to_9,per_5_to_9,10_to_14,per_10_to_14,15_to_19,per_15_to_19,20_to_24,per_20_to_24,25_to_29,per_25_to_29,30_to_34,per_30_to_34,35_to_39,per_35_to_39,40_to_44,per_40_to_44,45_to_49,per_45_to_49,50_to_54,per_50_to_54,55_to_59,per_55_to_59,60_to_64,per_60_to_64,65_to_69,per_65_to_69,70_to_74,per_70_to_74,75_to_79,per_75_to_79,80_to_84,per_80_to_89,80_and_over,per_80_and_over,disabled,per_disabled,unemployment_rate,tot_households_snap,households_snap,per_households_snap,tot_pop_mobility,same_house,moved_within_1yr,less_10k,10k_15k,15k_25k,25k_35k,35k_50k,50k_75k,75k_100k,100k_150k,150k_200k,more_200k,med_income,mean_incom,speaks_only_english,naturalized,non_citizen,pop_pov,pop_below_,pcnt_pov,families_on_suplimental_income,families_on_social_security,2_ppl_fam,3_to_4_ppl,5_to_6_ppl,GT_7_ppl_f,insured,per_insured,uninsured,per_uninsured,owner,rent,4_16_2020_positive,4_16_2020_tests,4_16_2020_positive_rate,4_2_2020_positive,4_2_2020_tests,4_2_2020_positive_rate,4_3_2020_positive,4_3_2020_tests,4_3_2020_positive_rate,4_7_2020_positive,4_7_2020_tests,4_7_2020_positive_rate,4_8_2020_positive,4_8_2020_tests,4_8_2020_positive_rate,hospital_count,3_26_bb_beds,4_2_bb_beds,4_7_bb_beds,licensed_beds,staffed_beds,ICU_beds,adult_icu_beds,pediatrics_icu_beds,bed_utilization_rate,potential_increase_bed_capacity,avg_ventilator_use
0,10065,984654,0,28109,24285,86.4,619,2.2,37,0.1,2666,9.5,0,0.0,154,0.5,348,1.2,12248,15861,1650,5.9,1000,3.6,916,3.3,356,1.3,1785,6.4,2671,9.5,3176,11.3,2558,9.1,1607,5.7,1570,5.6,1796,6.4,1460,5.2,1522,5.4,1388,4.9,1486,5.3,1159,4.1,954,3.4,1055,3.8,1894,6.8,70.2,14844,414,2.8,27635,22612,2247,5.0,2.7,4.0,2.7,6.2,10.6,8.8,14.4,9.8,35.8,127375,242978,19681,3045,3306,27963,1922,3.0,109,1683,3797,2267,275,13,27299,97.3,744,2.7,5365,9479,233,682,34.16,121,385,0.314286,121,385,0.314286,171,492,34.76,187,544,34.38,14,7625,7993,7993,6493,5519,745,745,402,6.647990,974,181
1,10069,249050,0,5085,3155,62.0,148,2.9,0,0.0,1558,30.6,0,0.0,0,0.0,224,4.4,2354,2731,368,7.2,416,8.2,274,5.4,95,1.9,235,4.6,427,8.4,594,11.7,639,12.6,508,10.0,374,7.4,378,7.4,235,4.6,197,3.9,61,1.2,87,1.7,109,2.1,0,0.0,88,1.7,182,3.6,74.7,2552,73,2.9,5025,3790,859,8.7,3.2,0.0,9.8,5.7,4.2,14.7,11.4,11.6,30.9,110625,225183,2922,331,1207,5085,522,7.3,25,88,517,662,27,0,4992,98.2,93,1.8,675,1877,40,105,38.10,24,57,0.421053,24,57,0.421053,29,72,40.28,32,81,39.51,9,4922,4922,4922,4093,3765,410,410,273,4.741397,328,138
2,10075,477137,0,21556,18396,85.3,677,3.1,225,1.0,1047,4.9,0,0.0,902,4.2,309,1.4,10096,11460,1041,4.8,1252,5.8,736,3.4,763,3.5,778,3.6,1400,6.5,2022,9.4,1643,7.6,1148,5.3,1453,6.7,1244,5.8,1118,5.2,1131,5.2,1561,7.2,1276,5.9,1169,5.4,570,2.6,1251,5.8,1689,8.0,68.1,11035,217,2.0,21381,18455,1508,1.9,0.8,4.8,6.4,5.0,12.8,10.6,17.9,9.6,30.3,137146,233358,13727,3315,2372,21155,1075,4.5,129,1401,3133,1765,236,0,20715,97.9,440,2.1,4933,6102,294,631,46.59,160,371,0.431267,160,371,0.431267,204,452,45.13,232,501,46.31,15,7573,7729,7729,6277,5678,788,788,412,7.253486,599,198
3,10128,1206191,0,59256,47167,79.6,2182,3.7,0,0.0,5844,9.9,0,0.0,1666,2.8,2397,4.0,25338,33918,3204,5.4,2138,3.6,1911,3.2,1691,2.9,2644,4.5,6825,11.5,7216,12.2,4349,7.3,3849,6.5,4103,6.9,3845,6.5,2989,5.0,3494,5.9,3461,5.8,2656,4.5,1980,3.3,1557,2.6,1344,2.3,4825,8.1,72.2,30691,1506,4.9,58419,48263,4921,4.5,2.2,6.5,3.6,5.6,12.4,11.2,14.6,10.0,29.4,114010,196844,39143,7054,6535,58980,3686,3.4,500,3513,8171,5192,530,3,57424,96.9,1820,3.1,10129,20562,460,1207,38.11,212,596,0.355705,212,596,0.355705,281,737,38.13,317,834,38.01,13,5936,6233,6233,4713,4177,514,514,337,6.464822,536,147
4,10280,297253,38409,9384,7360,78.4,184,2.0,0,0.0,1474,15.7,0,0.0,77,0.8,289,3.1,4477,4907,711,7.6,628,6.7,454,4.8,192,2.0,239,2.5,523,5.6,1374,14.6,945,10.1,1021,10.9,771,8.2,763,8.1,530,5.6,375,4.0,370,3.9,254,2.7,

In [11]:
boroughs_zip = pd.read_csv('data/bor_zip_codes.csv')
ny = pd.merge(ny,boroughs_zip[['GEOID10', 'borough']], left_on='GEOID_ZIP', right_on='GEOID10')
del(boroughs_zip)
ny.head()

,GEOID_ZIP,ALAND10,AWATER10,tot_pop,white,per_white,black,per_black,native,per_native,asian,per_asian,nativehawaiian,per_hawaiian,other,per_other,two_or_mor,per_two_or,male,female,under_5,per_under_5,5_to_9,per_5_to_9,10_to_14,per_10_to_14,15_to_19,per_15_to_19,20_to_24,per_20_to_24,25_to_29,per_25_to_29,30_to_34,per_30_to_34,35_to_39,per_35_to_39,40_to_44,per_40_to_44,45_to_49,per_45_to_49,50_to_54,per_50_to_54,55_to_59,per_55_to_59,60_to_64,per_60_to_64,65_to_69,per_65_to_69,70_to_74,per_70_to_74,75_to_79,per_75_to_79,80_to_84,per_80_to_89,80_and_over,per_80_and_over,disabled,per_disabled,unemployment_rate,tot_households_snap,households_snap,per_households_snap,tot_pop_mobility,same_house,moved_within_1yr,less_10k,10k_15k,15k_25k,25k_35k,35k_50k,50k_75k,75k_100k,100k_150k,150k_200k,more_200k,med_income,mean_incom,speaks_only_english,naturalized,non_citizen,pop_pov,pop_below_,pcnt_pov,families_on_suplimental_income,families_on_social_security,2_ppl_fam,3_to_4_ppl,5_to_6_ppl,GT_7_ppl_f,insured,per_insured,uninsured,per_uninsured,owner,rent,4_16_2020_positive,4_16_2020_tests,4_16_2020_positive_rate,4_2_2020_positive,4_2_2020_tests,4_2_2020_positive_rate,4_3_2020_positive,4_3_2020_tests,4_3_2020_positive_rate,4_7_2020_positive,4_7_2020_tests,4_7_2020_positive_rate,4_8_2020_positive,4_8_2020_tests,4_8_2020_positive_rate,hospital_count,3_26_bb_beds,4_2_bb_beds,4_7_bb_beds,licensed_beds,staffed_beds,ICU_beds,adult_icu_beds,pediatrics_icu_beds,bed_utilization_rate,potential_increase_bed_capacity,avg_ventilator_use,GEOID10,borough
0,10065,984654,0,28109,24285,86.4,619,2.2,37,0.1,2666,9.5,0,0.0,154,0.5,348,1.2,12248,15861,1650,5.9,1000,3.6,916,3.3,356,1.3,1785,6.4,2671,9.5,3176,11.3,2558,9.1,1607,5.7,1570,5.6,1796,6.4,1460,5.2,1522,5.4,1388,4.9,1486,5.3,1159,4.1,954,3.4,1055,3.8,1894,6.8,70.2,14844,414,2.8,27635,22612,2247,5.0,2.7,4.0,2.7,6.2,10.6,8.8,14.4,9.8,35.8,127375,242978,19681,3045,3306,27963,1922,3.0,109,1683,3797,2267,275,13,27299,97.3,744,2.7,5365,9479,233,682,34.16,121,385,0.314286,121,385,0.314286,171,492,34.76,187,544,34.38,14,7625,7993,7993,6493,5519,745,745,402,6.647990,974,181,10065,Manhattan
1,10069,249050,0,5085,3155,62.0,148,2.9,0,0.0,1558,30.6,0,0.0,0,0.0,224,4.4,2354,2731,368,7.2,416,8.2,274,5.4,95,1.9,235,4.6,427,8.4,594,11.7,639,12.6,508,10.0,374,7.4,378,7.4,235,4.6,197,3.9,61,1.2,87,1.7,109,2.1,0,0.0,88,1.7,182,3.6,74.7,2552,73,2.9,5025,3790,859,8.7,3.2,0.0,9.8,5.7,4.2,14.7,11.4,11.6,30.9,110625,225183,2922,331,1207,5085,522,7.3,25,88,517,662,27,0,4992,98.2,93,1.8,675,1877,40,105,38.10,24,57,0.421053,24,57,0.421053,29,72,40.28,32,81,39.51,9,4922,4922,4922,4093,3765,410,410,273,4.741397,328,138,10069,Manhattan
2,10075,477137,0,21556,18396,85.3,677,3.1,225,1.0,1047,4.9,0,0.0,902,4.2,309,1.4,10096,11460,1041,4.8,1252,5.8,736,3.4,763,3.5,778,3.6,1400,6.5,2022,9.4,1643,7.6,1148,5.3,1453,6.7,1244,5.8,1118,5.2,1131,5.2,1561,7.2,1276,5.9,1169,5.4,570,2.6,1251,5.8,1689,8.0,68.1,11035,217,2.0,21381,18455,1508,1.9,0.8,4.8,6.4,5.0,12.8,10.6,17.9,9.6,30.3,137146,233358,13727,3315,2372,21155,1075,4.5,129,1401,3133,1765,236,0,20715,97.9,440,2.1,4933,6102,294,631,46.59,160,371,0.431267,160,371,0.431267,204,452,45.13,232,501,46.31,15,7573,7729,7729,6277,5678,788,788,412,7.253486,599,198,10075,Manhattan
3,10128,1206191,0,59256,47167,79.6,2182,3.7,0,0.0,5844,9.9,0,0.0,1666,2.8,2397,4.0,25338,33918,3204,5.4,2138,3.6,1911,3.2,1691,2.9,2644,4.5,6825,11.5,7216,12.2,4349,7.3,3849,6.5,4103,6.9,3845,6.5,2989,5.0,3494,5.9,3461,5.8,2656,4.5,1980,3.3,1557,2.6,1344,2.3,4825,8.1,72.2,30691,1506,4.9,58419,48263,4921,4.5,2.2,6.5,3.6,5.6,12.4,11.2,14.6,10.0,29.4,114010,196844,39143,7054,6535,58980,3686,3.4,500,3513,8171,5192,530,3,57424,96.9,1820,3.1,10129,20562,460,1207,38.11,212,596,0.355705,212,596,0.355705,281,737,38.13,317,834,38.01,13,5936,6233,6233,4713,4177,514,514,337,6.464822,536,147,10128,Manhattan
4,10280,297253,38409,9384,7360,78.4,184,2.0,0,0.0,1474,15.7,0,0.0,77,0.8,289,3.1,4477,4907,711,7.6,628,6.7,454,4.8,192,2.0,239,2.5,523,5

In [12]:
gm = pd.read_csv('data/Google_Mobility_Report_Filtered.csv')
gm.head()

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,US,United States,New York,Bronx,2/15/2020,0,-8,-5,-3,0,1
1,US,United States,New York,Bronx,2/16/2020,-1,-4,5,-2,-2,0
2,US,United States,New York,Bronx,2/17/2020,3,-8,-11,-22,-38,10
3,US,United States,New York,Bronx,2/18/2020,-2,-6,-11,-7,-10,3
4,US,United States,New York,Bronx,2/19/2020,2,-6,0,-6,-9,2


In [13]:
# using an aggregate of drop in baseline from "normal" starting from Feb 15 - Apr 11
# figured an aggregate would be best way to deal with the abundance of data
gm_agg = gm.groupby(['sub_region_2']).agg({col: ['sum'] for col in ['retail_and_recreation_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline', 'parks_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline', 'workplaces_percent_change_from_baseline', 'residential_percent_change_from_baseline']})
gm_agg.columns = ['_'.join(multi_index) for multi_index in gm_agg.columns.ravel()]
gm_agg = gm_agg.reset_index()
gm_agg

,sub_region_2,retail_and_recreation_percent_change_from_baseline_sum,grocery_and_pharmacy_percent_change_from_baseline_sum,parks_percent_change_from_baseline_sum,transit_stations_percent_change_from_baseline_sum,workplaces_percent_change_from_baseline_sum,residential_percent_change_from_baseline_sum
0,Bronx,-1155,-211,-758,-1366,-1437,611
1,Brooklyn,-1407,-249,-45,-1677,-1601,715
2,Manhattan,-2296,-1006,-1544,-2199,-1926,777
3,Queens,-1470,-315,500,-1986,-1650,778
4,Staten Island,-1239,-254,-229,-1643,-1407,660


In [15]:
ny = pd.merge(ny, gm_agg, left_on='borough', right_on='sub_region_2')
ny.sample(10).head(10)

,GEOID_ZIP,ALAND10,AWATER10,tot_pop,white,per_white,black,per_black,native,per_native,asian,per_asian,nativehawaiian,per_hawaiian,other,per_other,two_or_mor,per_two_or,male,female,under_5,per_under_5,5_to_9,per_5_to_9,10_to_14,per_10_to_14,15_to_19,per_15_to_19,20_to_24,per_20_to_24,25_to_29,per_25_to_29,30_to_34,per_30_to_34,35_to_39,per_35_to_39,40_to_44,per_40_to_44,45_to_49,per_45_to_49,50_to_54,per_50_to_54,55_to_59,per_55_to_59,60_to_64,per_60_to_64,65_to_69,per_65_to_69,70_to_74,per_70_to_74,75_to_79,per_75_to_79,80_to_84,per_80_to_89,80_and_over,per_80_and_over,disabled,per_disabled,unemployment_rate,tot_households_snap,households_snap,per_households_snap,tot_pop_mobility,same_house,moved_within_1yr,less_10k,10k_15k,15k_25k,25k_35k,35k_50k,50k_75k,75k_100k,100k_150k,150k_200k,more_200k,med_income,mean_incom,speaks_only_english,naturalized,non_citizen,pop_pov,pop_below_,pcnt_pov,families_on_suplimental_income,families_on_social_security,2_ppl_fam,3_to_4_ppl,5_to_6_ppl,GT_7_ppl_f,insured,per_insured,uninsured,per_uninsured,owner,rent,4_16_2020_positive,4_16_2020_tests,4_16_2020_positive_rate,4_2_2020_positive,4_2_2020_tests,4_2_2020_positive_rate,4_3_2020_positive,4_3_2020_tests,4_3_2020_positive_rate,4_7_2020_positive,4_7_2020_tests,4_7_2020_positive_rate,4_8_2020_positive,4_8_2020_tests,4_8_2020_positive_rate,hospital_count,3_26_bb_beds,4_2_bb_beds,4_7_bb_beds,licensed_beds,staffed_beds,ICU_beds,adult_icu_beds,pediatrics_icu_beds,bed_utilization_rate,potential_increase_bed_capacity,avg_ventilator_use,GEOID10,borough,sub_region_2_x,retail_and_recreation_percent_change_from_baseline_sum_x,grocery_and_pharmacy_percent_change_from_baseline_sum_x,parks_percent_change_from_baseline_sum_x,transit_stations_percent_change_from_baseline_sum_x,workplaces_percent_change_from_baseline_sum_x,residential_percent_change_from_baseline_sum_x,sub_region_2_y,retail_and_recreation_percent_change_from_baseline_sum_y,grocery_and_pharmacy_percent_change_from_baseline_sum_y,parks_percent_change_from_baseline_sum_y,transit_stations_percent_change_from_baseline_sum_y,workplaces_percent_change_from_baseline_sum_y,residential_percent_change_from_baseline_sum_y
109,11214,5617496,8509,92946,45769,49.2,1364,1.5,192,0.2,34861,37.5,11,0.0,8279,8.9,2470,2.7,45577,47369,6466,7.0,5489,5.9,4961,5.3,4208,4.5,5229,5.6,6943,7.5,6618,7.1,7000,7.5,6263,6.7,6625,7.1,6284,6.8,6271,6.7,5238,5.6,5219,5.6,2891,3.1,2565,2.8,2290,2.5,2386,2.6,9557,10.3,61.1,31933,7451,23.3,91737,82013,7259,6.6,6.3,11.5,9.3,13.0,14.9,13.0,13.9,6.7,4.8,54041,74409,24057,31057,20759,92331,16168,13.7,2838,5784,7846,10595,3621,631,82189,88.9,10262,11.1,10629,21304,907,1599,56.72,251,544,0.461397,251,544,0.461397,403,796,50.63,523,996,52.51,1,371,481,481,371,292,22,22,10,0.798808,79,16,11214,Brooklyn,Brooklyn,-1407,-249,-45,-1677,-1601,715,Brooklyn,-1407,-249,-45,-1677,-1601,715
49,10306,19359797,152157,53142,45560,85.7,1775,3.3,36,0.1,3141,5.9,0,0.0,1278,2.4,1352,2.5,24906,28236,2308,4.3,2957,5.6,2962,5.6,3009,5.7,3230,6.1,3274,6.2,3640,6.8,2665,5.0,3386,6.4,3567,6.7,4359,8.2,4427,8.3,3897,7.3,2577,4.8,2433,4.6,1797,3.4,1196,2.3,1458,2.7,6311,12.0,58.8,20007,1836,9.2,52834,50520,1199,5.3,5.1,7.3,7.8,6.6,14.2,15.8,18.2,10.7,9.0,79569,96939,35191,8917,3366,52646,5191,8.4,916,4565,5450,6881,1620,169,50468,95.8,2198,4.2,14183,5824,1001,2285,43.81,278,632,0.439873,278,632,0.439873,404,884,45.70,513,1239,41.40,1,508,508,508,472,543,54,54,15,0.934081,-71,32,10306,Staten Island,Staten Island,-1239,-254,-229,-1643,-1407,660,Staten Island,-1239,-254,-229,-1643,-1407,660
22,10021,986790,0,46215,38017,82.3,420,0.9,158,0.3,4333,9.4,40,0.1,2069,4.5,1178,2.5,21010,25205,2869,6.2,2303,5.0,1760,3.8,950,2.1,880,1.9,3578,7.7,5750,12.4,4542,9.8,2628,5.7,2701,5.8,2797,6.1,3076,6.7,2131,4.6,2930,6.3,2275,4.9,1991,4.3,1721,3.7,1333,2.9,3867,8.4,68.9,24996,411,1.6,45496,37962,4154,5.3,1.2,4.0,3.8,5.6,12.6,11.5,13.2,9.8,33.0,118289,222872,33691,4849,4802,45697,2185,1.7,206,2747,5415,4248,365,125,43958,95.9,1902,4.

In [ ]:
# ny2 = pd.read_spss('data/NYCZCTA_2018_5YR_Estimate_FINAL_05052020.sav')
# ny2 = ny2.sort_values(by=['ZCTA'], ascending=[None])
# ny2.head()

## Data Transformations

In [ ]:
ny['per_minority'] =  ny['per_black'] + ny['per_native'] + ny['per_asian'] + ny['per_hawaiian'] + ny['per_other'] + ny['per_two_or'] 

In [ ]:
conditions = [(ny['per_minority'] <= 50.0),
              (ny['per_minority'] > 50.0)]
choices = [0,1]

ny['minority-majority-50'] = np.select(conditions, choices, default=2)

In [ ]:
conditions = [(ny['per_minority'] <= 60.0),
              (ny['per_minority'] > 60.0)]
choices = [0,1]

ny['minority-majority-60'] = np.select(conditions, choices, default=2)

In [ ]:
conditions = [(ny['per_minority'] <= 70.0),
              (ny['per_minority'] > 70.0)]
choices = [0,1]

ny['minority-majority-70'] = np.select(conditions, choices, default=2)

In [ ]:
old_columns = [x for x in ny.columns if x not in ['per_minority', 'minority-majority-50', 'minority-majority-60', 'minority-majority-70']]
insert_index = old_columns.index('4_16_2020_positive')
ny = ny[old_columns[:insert_index] + ['per_minority', 'minority-majority-50', 'minority-majority-60', 'minority-majority-70'] + old_columns[insert_index:]]

In [ ]:
ny["pop_density"] = ny['tot_pop']/ny['ALAND10']

In [ ]:
ny = ny[['GEOID_ZIP', 'ALAND10', 'AWATER10', 'tot_pop', 'white', 'per_white', 'black', 'per_black', 'native', 'per_native', 'asian', 'per_asian', 'nativehawaiian ', 'per_hawaiian', 'other', 'per_other', 'two_or_mor', 'per_two_or', 'male', 'female', 'under_5', 'per_under_5', '5_to_9', 'per_5_to_9', '10_to_14', 'per_10_to_14', '15_to_19', 'per_15_to_19', '20_to_24', 'per_20_to_24', '25_to_29', 'per_25_to_29', '30_to_34', 'per_30_to_34', '35_to_39', 'per_35_to_39', '40_to_44', 'per_40_to_44', '45_to_49', 'per_45_to_49', '50_to_54', 'per_50_to_54', '55_to_59', 'per_55_to_59', '60_to_64', 'per_60_to_64', '65_to_69', 'per_65_to_69', '70_to_74', 'per_70_to_74', '75_to_79', 'per_75_to_79', '80_to_84', 'per_80_to_89', '80_and_over', 'per_80_and_over', 'disabled', 'per_disabled', 'unemployment_rate', 'tot_households_snap', 'households_snap', 'per_households_snap', 'tot_pop_mobility', 'same_house', 'moved_within_1yr', 'less_10k', '10k_15k', '15k_25k', '25k_35k', '35k_50k', '50k_75k', '75k_100k', '100k_150k', '150k_200k', 'more_200k', 'med_income', 'mean_incom', 'speaks_only_english', 'naturalized', 'non_citizen', 'pop_pov', 'pop_below_', 'pcnt_pov', 'families_on_suplimental_income', 'families_on_social_security', '2_ppl_fam', '3_to_4_ppl', '5_to_6_ppl', 'GT_7_ppl_f', 'insured', 'per_insured', 'uninsured', 'per_uninsured', 'owner', 'rent', 'per_minority', 'minority-majority-50', 'minority-majority-60', 'minority-majority-70'] + ['pop_density'] + ['4_16_2020_positive', '4_16_2020_tests', '4_16_2020_positive_rate', '4_2_2020_positive', '4_2_2020_tests', '4_2_2020_positive_rate', '4_3_2020_positive', '4_3_2020_tests', '4_3_2020_positive_rate', '4_7_2020_positive', '4_7_2020_tests', '4_7_2020_positive_rate', '4_8_2020_positive', '4_8_2020_tests', '4_8_2020_positive_rate', 'hospital_count', '3_26_bb_beds', '4_2_bb_beds', '4_7_bb_beds', 'licensed_beds', 'staffed_beds', 'ICU_beds', 'adult_icu_beds', 'pediatrics_icu_beds', 'bed_utilization_rate', 'potential_increase_bed_capacity', 'avg_ventilator_use']]

## Initial OLS Model

In [ ]:
predictors = ["per_households_snap", "med_income", "pcnt_pov", "families_on_suplimental_income", "families_on_social_security", "per_uninsured", "minority-majority-60", "pop_density"]

X_train, X_test, y_train, y_test = train_test_split(ny[predictors], ny["4_16_2020_positive"], test_size=0.2)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

train_pred = model.predict(X_train)
test_pred = model.predict(X_test)


print('training r2 is:', model.score(X_train, y_train)) #training R2
print('testing r2 is:', model.score(X_test, y_test)) #testing R2

print('\ntrain mse is: ', mean_squared_error(y_train,train_pred))
print('test mse is: ', mean_squared_error(y_test,test_pred))

In [ ]:
ny.reset_index(inplace=True)
test_copy = X_test.copy()
test_copy.reset_index(inplace=True)

test_copy = pd.merge(test_copy, ny[['index','tot_pop','4_16_2020_positive']], on='index', how='left')
test_copy['ols_preds'] = test_pred
print(test_copy.head(25))

In [ ]:
coefficients = pd.DataFrame({"Coef":model.coef_,
              "Name": predictors})

coefficients = coefficients.append({"Coef": model.intercept_,
               "Name": "intercept"}, ignore_index = True)

print(coefficients)

In [ ]:
mod = sm.OLS(y_train,X_train)
fit = mod.fit()

In [ ]:
fit.summary()

In [ ]:
test_preds = fit.predict(X_test)

In [ ]:
#print('testing r2 is:', model.score(test_preds, y_test)) #testing R2